<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js109_electronPQM1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [ ]:
#@title js109_electronPQM1D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.12 created,  last updated on 2025.01.04
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js109_electronPQM1D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js109_electronPQM1D
//    Copyright(C) 2019-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2019.03.20 created, last updated on 2020.12.15
//    ver 0.0.1  2021.10.11 v1, last updated on 2021.10.12
//    ver 0.0.2  2021.11.07 v2, last updated on 2021.11.07
//    ver 0.0.3  2023.05.17 v3, last updated on 2023.09.12
//
//
// --------------------  PQM: Periodic Quantum Mechanics 1D (explecit method)
//
// - time dependent Schrodinger equation: i(d/dt)psi(r,t) = H psi(r,t)
//
// - time evolution
//    psi(r,t+dt) = exp(-i dt H) psi(r,t),  (H:Hamiltonian of the system)
//      H = -delta/2 + V(r), delta = d^2/dx^2
//    psi(r,t+dt) = exp(-i dt H) psi(r,t)
//                ~ {exp(-i(dt/2)V} {exp(i dt(delta/2)} {exp(-i(dt/2)V} psi(r,t)
//
//      {exp(-i(dt/2)V} psi(r,t): phase rotate theta = (dt/2)*V
//
//      {exp(i dt(delta/2)}: Richardson algorithm - explecit method
//        Richardson; Computer Physics Communications vol.63 page 84-94 (1991)
//
// --------------------
*/

const electronPQM1D = (function(){ // ====================  electronPQM1D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;		// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;		// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;		// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;				// (eV) 1(au) = 27.211386 (eV)

	let g_NNx = 400;						// number of space division (g_NNx*g_dx = x-length)
	let g_sysTime = 0.0;					// (au) system time
	let g_dx = 0.5;							// (au) space division
	let g_timeStep = 0.1*g_dx*g_dx;			// (au) time step dt
	let dampingFactor = 0.1;				// for lossEnergy() damping factor :steepest descent method

	const g_psi = dim2( g_NNx, 2 );			// Re(g_psi) = g_psi[i][0], Im(g_psi) = g_psi[i][1]
	const g_vv = dim1( g_NNx );				// external potential
	const g_wrkRe = dim1( g_NNx );			// for lossEnergy() real part of work vector
	const g_wrkIm = dim1( g_NNx );			// for lossEnergy() imaginary part of work vector

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		const xMax=g_NNx*g_dx;

		g_sysTime = 0.0;
		if (theme==0) { // 0:free space
			setFreePotential();
			setWave(0.25*xMax, 10.0, 1.0); // (wavePos,waveWidth,momentum), K=momentum^2/(2me) = 0.5
		} else if (theme==1) { // 1:parabolic potential
			setParabolicPotential(0.5*xMax, 0.0001); // (xpos,k)
			setWave(0.25*xMax, 10.0, 0.0); // (wavePos,waveWidth,momentum)
		} else if (theme==2) { // 2:tunnel effect
			setWallPotential(0.5*xMax, 10*g_dx, 0.55); // (xpos,width,hight)
			setWave(0.25*xMax, 10.0, 1.0); // (wavePos,waveWidth,momentum), K=momentum^2/(2me) = 0.5
		} else if (theme==3) { // 3:step hill
			setWallPotential(0.5*xMax, 0.5*xMax, 0.3); // (xpos,width,hight)
			setWave(0.25*xMax, 10.0, 1.0); // (wavePos,waveWidth,momentum), K=momentum^2/(2me) = 0.5
		} else if (theme==4) { // 4:step higher hill
			setWallPotential(0.7*xMax, 0.5*xMax, 0.8); // (xpos,width,hight)
			setWave(0.25*xMax, 10.0, 1.0); // (wavePos,waveWidth,momentum), K=momentum^2/(2me) = 0.5
		}
	}

	function setFreePotential() {
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			g_vv[i] = 0.0;
		}
	}

	function setParabolicPotential(xpos,k) {
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			g_vv[i] = k*(x-xpos)*(x-xpos);
		}
	}

	function setWallPotential(xpos,width,hight) {
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			g_vv[i] = (x>=xpos && x<xpos+width) ? hight : 0.0;
		}
	}

	function setWave(wavePos,waveWidth,momentum) {
		const nnx=g_NNx;

		const a = Math.pow(2.0*Math.PI*waveWidth*waveWidth,-0.25);
		for (let i=1; i<nnx-1; i++) {
			const x = i*g_dx;
			const phAb = a*Math.exp(-((x-wavePos)/(2.0*waveWidth))*((x-wavePos)/(2.0*waveWidth)));
			const phPh = momentum*x;
			g_psi[i][0] = phAb*Math.cos(phPh);
			g_psi[i][1] = phAb*Math.sin(phPh);
		}
		g_psi[0][0] = 0.0;
		g_psi[0][1] = 0.0;
		g_psi[nnx-1][0] = 0.0;
		g_psi[nnx-1][1] = 0.0;

		normalize();
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( lossSW ) {

		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
			kxStep(g_psi,g_timeStep);
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
		}
		if (lossSW==1) lossEnergy(dampingFactor);
	}

	function kxStep(ph, deltat) {
		const nnx=g_NNx;
		const epsilonX = deltat/(g_dx*g_dx);
		const alphaXRe = 0.5 * (1.0+Math.cos(epsilonX/2));
		const alphaXIm = -0.5 * Math.sin(epsilonX/2);
		const betaXRe = Math.sin(epsilonX/4)*Math.sin(epsilonX/4);
		const betaXIm = 0.5 * Math.sin(epsilonX/2);

		for (let i=0; i<nnx-1; i+=2) {
			substepX(ph,i,i+1);
		}
		for (let i=1; i<nnx-1; i+=2) {
			substepX(ph,i,i+1);
		}
		substepX(ph,nnx-1,0); // periodic-x condition

		substepX(ph,nnx-1,0); // periodic-x condition
		for (let i=1; i<nnx-1; i+=2) {
			substepX(ph,i,i+1);
		}
		for (let i=0; i<nnx-1; i+=2) {
			substepX(ph,i,i+1);
		}

		function substepX(ph,i,i2) {
			const xRe = ph[i][0], xIm = ph[i][1];
			const yRe = ph[i2][0], yIm = ph[i2][1];
			ph[i][0] = (alphaXRe*xRe-alphaXIm*xIm) + (betaXRe*yRe-betaXIm*yIm);
			ph[i][1] = (alphaXRe*xIm+alphaXIm*xRe) + (betaXRe*yIm+betaXIm*yRe);
			ph[i2][0] = (alphaXRe*yRe-alphaXIm*yIm) + (betaXRe*xRe-betaXIm*xIm);
			ph[i2][1] = (alphaXRe*yIm+alphaXIm*yRe) + (betaXRe*xIm+betaXIm*xRe);
		}
	}

	function phaseStep(ph, vv, deltat) { // ph[][], vv[]
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			const th = deltat*vv[i];
			const costh = Math.cos(th), sinth = Math.sin(th);
			const phRe = ph[i][0], phIm = ph[i][1];
			ph[i][0] = costh*phRe + sinth*phIm;
			ph[i][1] =-sinth*phRe + costh*phIm;
		}
	}

	function lossEnergy(damp) { // steepest descent method: psi_next = psi - damp*(psi|H-E|psi)
		const nnx=g_NNx, h2=2.0*g_dx*g_dx;
		const psi=g_psi,wrkRe=g_wrkRe, wrkIm=g_wrkIm;

		const ee = kineticEnergy() + potentialEnergy();
		for (let i=0; i<nnx; i++) { // wrk = (psi(x)|-d^2/dx^2+V-ee|psi(x))
			const ipp = (i+1)%nnx, imm = (i-1+nnx)%nnx;
			wrkRe[i] = -(psi[ipp][0]+psi[imm][0]-2.0*psi[i][0])/h2+(g_vv[i]-ee)*psi[i][0];
			wrkIm[i] = -(psi[ipp][1]+psi[imm][1]-2.0*psi[i][1])/h2+(g_vv[i]-ee)*psi[i][1];
		}
		for (let i=0; i<nnx; i++) { // ph = ph -damp*(psi(x)|-d^2/dx^2+V-ee|psi(x))
			psi[i][0] -= damp*wrkRe[i];
			psi[i][1] -= damp*wrkIm[i];
		}
		normalize();
	}

	// --------------------  utility  --------------------

	function norm() {
		const nnx=g_NNx, psi=g_psi;

		let p = 0.0;
		for (let i=0; i<nnx; i++) {
			p += (psi[i][0]*psi[i][0]+psi[i][1]*psi[i][1])*g_dx;
		}
		return p;
	}

	function normalize() {
		const nnx=g_NNx, psi=g_psi;

		const a = Math.sqrt(norm());
		for (let i=0; i<nnx; i++) {
			psi[i][0] = psi[i][0]/a;
			psi[i][1] = psi[i][1]/a;
		}
	}

	function energy() {
		return kineticEnergy()+potentialEnergy();
	}

	function kineticEnergy() { // (psi(x)|-d^2/dx^2|psi(x))
		const nnx=g_NNx, psi=g_psi;

		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			const ipp = (i+1)%nnx, imm = (i-1+nnx)%nnx;
			const hphRe = (2.0*psi[i][0]-psi[ipp][0]-psi[imm][0])/(2.0*g_dx*g_dx);
			const hphIm = (2.0*psi[i][1]-psi[ipp][1]-psi[imm][1])/(2.0*g_dx*g_dx);
			s += (psi[i][0]*hphRe + psi[i][1]*hphIm)*g_dx;
		}
		return s;
	}

	function potentialEnergy() { // (psi(x)|V(x)|psi(x))
		const nnx=g_NNx, psi=g_psi;

		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			s += g_vv[i]*(psi[i][0]*psi[i][0]+psi[i][1]*psi[i][1])*g_dx;
		}
		return s;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( lossSW )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_dx, g_timeStep ]; },
		getNow:			function() { return [ g_sysTime, norm(), kineticEnergy(), potentialEnergy() ]; },

		getPsi:			function(i) { return [ g_psi[i][0], g_psi[i][1] ]; },
		getPsiDensity:	function(i) { return (g_psi[i][0]*g_psi[i][0] + g_psi[i][1]*g_psi[i][1]); },
		getVext:		function(i) { return g_vv[i]; },

	}

})(); // ====================  electronPQM1D end  ====================


const js109 = (function(){ // ====================  js Module  ====================

	const theModule = electronPQM1D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 0;			// 0:free space, 1:parabolic potential, 2:tunnel effect 3:step hill 4:step higher hill
	let v_lossSW = 0;			// if lossSW==1, call lossEnergy()

	let p_NNx, p_dx, p_timeStep; // = theModule.getSysParam();
	let p_NNy, p_NNz;

  let sysTime, norm, kineticEnergy, potentialEnergy;
	let nowData = [];
  let vextList = [];
  let psiReList = [];
  let psiImList = [];

	let dispMode = 2;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_NNx, p_dx, p_timeStep ] = theModule.getSysParam();
			p_NNy = p_NNx, p_NNz = p_NNx;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			v_lossSW = 0;
			document.getElementById("slct_loss").selectedIndex = 0;
      getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_lossSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_lossSW );
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
      vextList = [];
      psiReList = [];
      psiImList = [];
      nowData = [ sysTime, norm, kineticEnergy, potentialEnergy ];
      for (let i=0; i<p_NNx; i++) {
        vextList[i] = theModule.getVext(i);
        let pRe, pIm;
        [ pRe, pIm ] = theModule.getPsi(i);
        psiReList[i] = pRe;
        psiImList[i] = pIm;
      }
    }
  }


	// --------------------  draw  --------------------

	const gColor = { psi:'#dd00dd', dens:'#dd88dd', Vext:'#00ff00', text:'#999999' };

	function draw( ctx, dispMode ) {
		const xTextPos = 20, ytextPos = yCanvasSize-45;

		let auLength, auTime, auEnergy, au2eV;
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ sysTime, norm, kineticEnergy, potentialEnergy ] = theModule.getNow();
		const totalEnergy = kineticEnergy + potentialEnergy;

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if ( dispMode==0 ) { // 2D psi(x), |psi(x)|^2, Vext(x)
			const xBoxPos = 40, yBoxPos = 40, xBoxSize = 400, yBoxSize = 340, ytextPos = 420;
			const xp = 40, yp = 240, vmag = 100.0, rmag = 2000.0, pmag = 100.0;

			// box
			ctx.strokeStyle = "#888800";
			ctx.strokeRect( xBoxPos, yBoxPos, xBoxSize, yBoxSize );

			// draw wave, |psi(x)|^2, Vext(x)
			drawV( ctx, p_NNx, xp, yp, vmag, gColor.Vext );
			drawPsi2( ctx, p_NNx, p_dx, xp, yp, vmag, rmag, gColor.dens );
			drawWave( ctx, p_NNx, totalEnergy, xp, yp, vmag, pmag );

			ctx.fillStyle = gColor.dens;
			ctx.fillText("|psi(x)|^2", xp, 25);
			ctx.fillStyle = gColor.psi;
			ctx.fillText("psi(x)", xp+170, 25);
			ctx.fillStyle = gColor.Vext;
			ctx.fillText("Vext(x)", xp+320, 25);

		} else if ( dispMode==1 ) { // 3D |psi(x)|^2, Vext(x)
			// drawLines3D( ctx, mode, totalEnergy )
			drawLines3D( ctx, 0, totalEnergy );

		} else if ( dispMode==2 ) { // 3D wave, Vext(x)
			// drawLines3D( ctx, mode, totalEnergy )
			drawLines3D( ctx, 1, totalEnergy );

		}

		// caption
		ctx.fillStyle = gColor.text;
		ctx.fillText(`time = ${(sysTime*auTime*1.0e15).toFixed(2)} (fs)`, xTextPos, ytextPos);
		ctx.fillText(`norm = ${norm.toFixed(15)}`, xTextPos+200, ytextPos);
		ctx.fillText(`kineticE = ${(kineticEnergy*au2eV).toFixed(3)} (eV)`, xTextPos, ytextPos+20);
		ctx.fillText(`potentialE = ${(potentialEnergy*au2eV).toFixed(3)} (eV)`, xTextPos+200, ytextPos+20);
		ctx.fillText(`totalE= ${(totalEnergy*au2eV).toFixed(3)} (eV)`, xTextPos, ytextPos+40);
		ctx.fillText(`x = 0.0 ... ${(p_NNx*p_dx*auLength*1.0e9).toFixed(2)} (nm)`, xTextPos+200, ytextPos+40);


		function drawPsi2( ctx, nnx, dx, xp, yp, vmag, rmag, color ) {

			for (let i=1; i<nnx-1; i++) {
				let phRe, phIm;
				[ phRe, phIm ] = theModule.getPsi(i);
				const p2 = (phRe*phRe + phIm*phIm)*dx;
				if (p2*rmag>1) {
					const vvi = theModule.getVext(i);
					drawLine( ctx, i+xp,yp-vvi*vmag-1, i+xp, yp-vvi*vmag-p2*rmag, color);
				}
			}
		}

		function drawV( ctx, nnx, xp, yp, vmag, color ) {

			ctx.strokeStyle = color;
			ctx.beginPath();
			for (let i=0; i<nnx-1; i++) {
				const vvi = theModule.getVext(i);
				ctx.lineTo(i+xp,yp-vvi*vmag);
			}
			ctx.stroke();
		}

		function drawWave( ctx, nnx, totalEnergy, xp, yp, vmag, pmag ) {
			const thInDegree = 15.0, deg = 180.0/Math.PI;

			const snth = Math.sin(thInDegree/deg);
			const csth = Math.cos(thInDegree/deg);
			for (let i=0; i<nnx-1; i++) {
				let phRe, phIm;
				[ phRe, phIm ] = theModule.getPsi(i);
				const ph0 = phRe*pmag;
				const ph1 = phIm*pmag;
				ctx.strokeStyle = "hsl("+(Math.atan2(ph1,ph0)*deg)+",100%,50%)";
				ctx.beginPath();
				ctx.moveTo(xp+(i+snth*ph1),yp-(ph0+csth*ph1)-totalEnergy*vmag);
				[ phRe, phIm ] = theModule.getPsi(i+1);
				const ph01 = phRe*pmag;
				const ph11 = phIm*pmag;
				ctx.lineTo(xp+(i+snth*ph11),yp-(ph01+csth*ph11)-totalEnergy*vmag);
				ctx.stroke();
			}
		}

		function drawLine(ctx, x1, y1, x2, y2, color) {
			ctx.strokeStyle = color;
			ctx.beginPath();
			ctx.moveTo(x1, y1);
			ctx.lineTo(x2, y2);
			ctx.stroke();
		}
	}


	function drawLines3D( ctx, mode, totalEnergy ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, degree = 180.0/Math.PI;
		const vmag = 50.0, dmag = 500.0, pmag = 100.0;

		const densVextFunc = function(i) { return vmag*theModule.getVext(i) + dmag*theModule.getPsiDensity(i); };

		g3d.set3DRotateXY(0.0);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow

		if ( mode==0 ) { // |psi(x)|^2(x) + Vext(x)
			yPlot( ctx, densVextFunc, 1.0, gColor.dens );
			yPlot( ctx, theModule.getVext, vmag, gColor.Vext );
			ctx.fillStyle = gColor.dens;
			ctx.fillText(`density(x)`, 300, 20);
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 400, 20);

		} else if ( mode==1 ) { // psi: y:Re[psi(x)], z:Im[psi(x)]
			yPlot( ctx, theModule.getVext, vmag, gColor.Vext );
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 400, 20);

			const yConst = vmag*totalEnergy;
			for (let i=0; i<nnx-1; i++) {
				let yi, zi, yip, zip;
				[ yi, zi ] = theModule.getPsi(i);
				const x = i*g_dx, y = g3d.cy0-pmag*yi-yConst, z = g3d.cz0+pmag*zi;
				[ yip, zip ] = theModule.getPsi(i+1);
				const x1 = (i+1)*g_dx, y1 = g3d.cy0-pmag*yip-yConst, z1 = g3d.cz0+pmag*zip;
				const amp = pmag*Math.sqrt(yi*yi+zi*zi + yip*yip+zip*zip);
				const th = Math.atan2(zi+zip,yi+yip)*degree+180.0;
				const colr = (amp>0.5) ? `hsl(${th},100%,50%)` : `#444444`;
				g3d.drawRotatedLine( ctx, x, y, z, x1, y1, z1, colr, sc, xp, yp );
			}
			ctx.fillStyle = gColor.psi;
			ctx.fillText(`psi(x)`, 300, 20);
		}

		g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";

		function yPlot( ctx, yFunc, mag, color ) {
			let yi, yip;
			for (let i=0; i<nnx-1; i++) {
				yi = mag*yFunc( i );
				const x = i*g_dx, y = g3d.cy0-yi, z = g3d.cz0;
				yip = mag*yFunc( i+1 );
				const x1 = (i+1)*g_dx, y1 = g3d.cy0-yip, z1 = g3d.cz0;
				g3d.drawRotatedLine(ctx, x, y, z, x1, y1, z1, color, sc, xp, yp );
			}
		}
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		const dom = document.getElementById("slct_orbit");
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-10,-10); // in degree
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, norm, kineticEnergy, potentialEnergy ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, vextList, psiReList, psiImList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setLossSW:		setLossSW,		// setLossSW()

		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, norm, kineticEnergy, potentialEnergy ]
    pygetFieldData: pygetFieldData, // pygetFieldData() : return [ nowData, vextList, psiReList, psiImList ]
	};

})(); // ====================  js109 module end  ====================


const js = js109;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js109] electron wave packet in the periodic box PQM1D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas><br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>in free space</option><option>in parabollic V=k r^2</option>
<option>tunnel effect</option><option>step hill</option><option>step higher hill</option></select>
    <span style="margin-right: 20px;"></span>
<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option selected>off</option><option>on</option></select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>2D: wave, |psi(x)|^2, Vext(x)</option>
<option>3D: |psi(x)|^2, Vext(x)</option>
<option selected>3D: wave, Vext(x)</option>
</select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f'i = {i:>2},  time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme, dispMode

import time

themeList = [ '0: in free space', '1: in parabollic V=k r^2', '2: tunnel effect', '3: step hill', '4: step higher hill' ]
dispModeList = [ '0: 2D - wave, |psi(x)|^2, Vext(x)', '1: 3D - |psi(x)|^2, Vext(x)', '2; 3D - wave, Vext(x)' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: in parabollic V=k r^2'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# chabge disp mode and get data and print
for dispMode in range(len(dispModeList)):
  # change dispMode
  print(f"  -- dispMode: {dispModeList[dispMode]} --")
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )

  # get data and print
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f'\t time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / total energy ~ constant

import time
import numpy as np
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: in parabollic V=k r^2'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# get K U data
timeList = []
kkList = []
uuList = []
energyList = []

for i in range(50):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  if i%5==0:
    print(f"\t time = {sysTime:>8.2f} (au),  energy(K+U) = {energy:>8.6f} (au),  K = {kineticEnergy:>8.6f} (au),  U = {potentialEnergy:>8.6f} (au)")

  timeList.append(sysTime)
  kkList.append(kineticEnergy)
  uuList.append(potentialEnergy)
  energyList.append(energy)

  time.sleep(0.5)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

print("-- plot energy(blue), kinetic energy(red), potential energy(green) --")
plt.plot( timeList, kkList, 'or-' ) # kinetic energy red
plt.plot( timeList, uuList, '^g-' ) # potential energy green
plt.plot( timeList, energyList, 'xb-' ) # total energy blue
plt.xlabel("time (au)")
plt.ylabel("energy (au)")
plt.show()

In [ ]:
# get field data: vextList, psiReList, psiImList and save

import time
import numpy as np

themeList = [ '0: in free space', '1: in parabollic V=k r^2', '2: tunnel effect', '3: step hill', '4: step higher hill' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: in parabollic V=k r^2'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# get data and print
for i in range(10):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f'i = {i:>2},  time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
  time.sleep(1)

# get data  spinorRe, spinorIm --> SpRe, SpIm
print("-- get spinor data --")
[ nowData, vextList, psiReList, psiImList ] = eval_js( 'js.pygetFieldData()' )
[ sysTime, norm, kineticEnergy, potentialEnergy ] = nowData
print(f'got time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')

# convert to numpy array
Vext = np.array(vextList)
PsiRe = np.array(psiReList)
PsiIm = np.array(psiImList)
print(f' Vext shape :{Vext.shape},  PsiRe shape :{PsiRe.shape},  PsiIm shape :{PsiIm.shape}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# save data
print("-- save data : np.array([ vextList, psiReList, psiImList ]) --")
np_data = np.array([ vextList, psiReList, psiImList ])

np.save('js109_data.npy', np_data)
print("-- saved data shape :", np_data.shape, " --")

In [ ]:
# load data and set numpy array for plot

import numpy as np

# load data : loaded_data <-- np.array([ vextList, psiReList, psiImList ])
loaded_data = np.load('js109_data.npy')

# set numpy array
Vext = loaded_data[0]
PsiRe = loaded_data[1]
PsiIm = loaded_data[2]
print(f' Vext shape :{Vext.shape},  PsiRe shape :{PsiRe.shape},  PsiIm shape :{PsiIm.shape}')

 Vext shape :(400,),  PsiRe shape :(400,),  PsiIm shape :(400,)


In [ ]:
# plot Vext(x)

import numpy as np
import matplotlib.pyplot as plt

nx = len(Vext)
h = 0.5
X = np.linspace(-nx/2*h, nx/2*h, nx)
plt.plot(X,Vext)

In [ ]:
# plot Re[Psi(x)], Im[Psi(x)], Vext(x)

import numpy as np
import matplotlib.pyplot as plt

nx = len(PsiRe)
h = 0.5
X = np.linspace(-nx/2*h, nx/2*h, nx)

plt.plot(X, Vext*0.2, 'g-' )
plt.plot(X, PsiRe, 'r-')
plt.plot(X, PsiIm, 'b-')
plt.title("electron red:Re[Psi(x)], blue:Im[Psi(x)], green:Vext(x) x 0.2")
plt.show()

In [ ]:
# plot  Vext(x), Psi(x) - plotly

import plotly.graph_objects as go
import numpy as np

# X
nx = len(Vext)
h = 0.5
X = np.linspace(-nx/2*h, nx/2*h, nx)

# plot Psi(x)
Y = PsiRe
Z = PsiIm
fig = go.Figure(data=go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name='Psi(x)'))

# plot Vext(x)
Y = Vext*0.2
Z = np.full_like(PsiRe, 0)
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name='Vext(x)'))

fig.update_layout(
    title='Psi(x) and Vext(x)',
    width=1000, height=700,
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode="manual",
        aspectratio=dict(x=1, y=0.5, z=0.5),  # set aspect ratio
        camera=dict(
            eye=dict(x=0, y=0.5, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=1, z=0)  # upside vector
        )
    )
)

fig.show()

In [ ]:
# prompt: plot Phase (= atan2(PsiIm, PsiRe)*180/pi) * Abs(Psi)

import numpy as np
import matplotlib.pyplot as plt

# Assuming you have already loaded your data as Vext, PsiRe, and PsiIm

# Calculate Phase and Abs(Psi)
Psi = PsiRe + 1j * PsiIm
Phase = np.arctan2(PsiIm, PsiRe) * 180 / np.pi
AbsPsi = np.abs(Psi)

# Calculate the desired quantity
Y = Phase * AbsPsi

# X
nx = len(Vext)
h = 0.5
X = np.linspace(-nx/2*h, nx/2*h, nx)


plt.plot(X, Y)
plt.xlabel("X")
plt.ylabel("Phase * Abs(Psi)")
plt.title("Plot of Phase * Abs(Psi)")
plt.show()

In [ ]:
# prompt: calc sum of (psiRe**2 + psiIm**2)*h
# = norm of Psi

# Assuming you have already loaded your data as Vext, PsiRe, and PsiIm
# and h is defined as 0.5

h = 0.5  # Make sure h is defined
sum_val = np.sum((PsiRe**2 + PsiIm**2) * h)

print(f"The sum of (psiRe**2 + psiIm**2) * h is: {sum_val}")